# Quick audit of `metadata_wav.csv`

Goals:
- load metadata;
- inspect shapes, dtypes, and unique values of key fields;
- find missing values and obvious duplicates;
- review duration and sample rate distributions;
- verify that WAV file paths exist.

In [12]:
import os
import pandas as pd
from pathlib import Path
import numpy as np

pd.options.display.max_rows = 20
pd.options.display.max_columns = None

DATA_PATH = Path("/Volumes/SSanDisk/SpeechRec-German/data_wav/metadata_wav.csv")
DATA_PATH

PosixPath('/Volumes/SSanDisk/SpeechRec-German/data_wav/metadata_wav.csv')

In [13]:
df = pd.read_csv(DATA_PATH)
df.shape

(39248, 12)

In [14]:
# Basic info and a few rows for orientation
display(df.head())
df.info()

,idx,id,subset,style,text,samplerate,durationSeconds,recording_year_month,microphone,language,comment,audio_wav_path
0,0,4aeeae88-0777-2c8c-5c93-2e844a462e49---0e52cfa...,TV-2021.02-Neutral,neutral,"Man könnte sagen, ich sei für diese Aufgabe pr...",16000,4.000000,2019-11,bad_usbHeadset,german,NaN,/Volumes/SSanDisk/SpeechRec-German/data_wav/TV...
1,1,4aeeae88-0777-2c8c-5c93-2e844a462e49---2a5f795...,TV-2021.02-Neutral,neutral,Das heutige vereinte System von Postleitzahlen...,16000,7.851562,2020-02,good_rodePodcaster,german,NaN,/Volumes/SSanDisk/SpeechRec-German/data_wav/TV...
2,2,4aeeae88-0777-2c8c-5c93-2e844a462e49---6dba565...,TV-2021.02-Neutral,neutral,Eine komplizierte Story mit unzähligen Charakt...,16000,5.750000,2020-02,good_rodePodcaster,german,NaN,/Volumes/SSanDisk/SpeechRec-German/data_wav/TV...
3,3,4aeeae88-0777-2c8c-5c93-2e844a462e49---9d651de...,TV-2021.02-Neutral,neutral,Siehe weiter unten.,16000,1.540039,2019-12,bad_usbHeadset,german,NaN,/Volumes/SSanDisk/SpeechRec-German/data_wav/TV...
4,4,4aeeae88-0777-2c8c-5c93-2e844a462e49---9c33fb6...,TV-2021.02-Neutral,neutral,Bei niedriger Last werden bis zu vier der acht...,16000,5.218750,2019-10,bad_usbHeadset,german,NaN,/Volumes/SSanDisk/SpeechRec-German/data_wav/TV...


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39248 entries, 0 to 39247
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   idx                   39248 non-null  int64  
 1   id                    39248 non-null  object 
 2   subset                39248 non-null  object 
 3   style                 39248 non-null  object 
 4   text                  39248 non-null  object 
 5   samplerate            39248 non-null  int64  
 6   durationSeconds       39248 non-null  float64
 7   recording_year_month  39248 non-null  object 
 8   microphone            39248 non-null  object 
 9   language              39248 non-null  object 
 10  comment               2020 non-null   object 
 11  audio_wav_path        39248 non-null  object 
dtypes: float64(1), int64(2), object(9)
memory usage: 3.6+ MB


In [15]:
# Quick stats for numeric columns
df.describe().T

,count,mean,std,min,25%,50%,75%,max
idx,39248.0,19623.500000,11330.066019,0.00000,9811.750000,19623.500000,29435.250000,39247.000000
samplerate,39248.0,16000.000000,0.000000,16000.00000,16000.000000,16000.000000,16000.000000,16000.000000
durationSeconds,39248.0,3.565031,1.831246,0.22998,2.320312,3.279297,4.351562,13.523438


In [16]:
# Missing values and empty strings (for object columns, treat blank/whitespace as missing)
str_cols = df.select_dtypes(include="object").columns
empty_counts = df[str_cols].apply(lambda s: (s.fillna("").str.strip() == "").sum())
null_counts = df.isna().sum()
pd.DataFrame({"null": null_counts, "empty_strings": empty_counts}).sort_values(by=["null", "empty_strings"], ascending=False)

,null,empty_strings
comment,37228,37228.0
audio_wav_path,0,0.0
id,0,0.0
language,0,0.0
microphone,0,0.0
recording_year_month,0,0.0
style,0,0.0
subset,0,0.0
text,0,0.0
durationSeconds,0,NaN


In [17]:
# Duplicate checks on key fields
dup_id = df[df.duplicated(subset=["id"], keep=False)]
dup_path = df[df.duplicated(subset=["audio_wav_path"], keep=False)]
dup_text = df[df["text"].str.strip().duplicated(keep=False)]

print("Duplicates by id:", len(dup_id))
print("Duplicates by audio_wav_path:", len(dup_path))
print("Duplicates by text (trimmed):", len(dup_text))

# Show top matches for diagnosis
dup_id.head(), dup_path.head(), dup_text.head()

Duplicates by id: 0
Duplicates by audio_wav_path: 0
Duplicates by text (trimmed): 5743


(Empty DataFrame
 Columns: [idx, id, subset, style, text, samplerate, durationSeconds, recording_year_month, microphone, language, comment, audio_wav_path]
 Index: [],
 Empty DataFrame
 Columns: [idx, id, subset, style, text, samplerate, durationSeconds, recording_year_month, microphone, language, comment, audio_wav_path]
 Index: [],
     idx                                                 id  \
 32   32  4aeeae88-0777-2c8c-5c93-2e844a462e49---3c316f5...   
 42   42  4aeeae88-0777-2c8c-5c93-2e844a462e49---5101565...   
 78   78  4aeeae88-0777-2c8c-5c93-2e844a462e49---a2e0294...   
 80   80  4aeeae88-0777-2c8c-5c93-2e844a462e49---26e059b...   
 81   81  4aeeae88-0777-2c8c-5c93-2e844a462e49---d3d6f74...   
 
                 subset    style  \
 32  TV-2021.02-Neutral  neutral   
 42  TV-2021.02-Neutral  neutral   
 78  TV-2021.02-Neutral  neutral   
 80  TV-2021.02-Neutral  neutral   
 81  TV-2021.02-Neutral  neutral   
 
                                                  text  samplerate

In [18]:
# Categorical value distributions
summary = {
    "subset": df["subset"].value_counts(),
    "style": df["style"].value_counts(),
    "language": df["language"].value_counts(),
    "microphone": df["microphone"].value_counts(),
    "sample_rates": df["samplerate"].value_counts(),
}
for name, series in summary.items():
    print(f"\n{name} (top 10):")
    display(series.head(10))


subset (top 10):


subset
TV-2021.02-Neutral      22671
TV-2022.10-Neutral      12451
TV-2023.09-Hessisch      2106
TV-2021.06-Emotional     2020
Name: count, dtype: int64


style (top 10):


style
neutral                      35122
Hessisch (german dialect)     2106
angry | wütend                 300
amused | amüsiert              300
disgusted | angewidert         300
drunk | angetrunken            300
surprised | überrascht         300
whisper | flüstern             299
sleepy | schläfrig             221
Name: count, dtype: int64


language (top 10):


language
german    39248
Name: count, dtype: int64


microphone (top 10):


microphone
good_rodePodcaster    24528
bad_usbHeadset        14720
Name: count, dtype: int64


sample_rates (top 10):


samplerate
16000    39248
Name: count, dtype: int64

In [19]:
# Duration ranges and count of zero/negative values
dur = df["durationSeconds"]
print("min", dur.min(), "max", dur.max(), "mean", dur.mean())
print("<= 0 durations:", (dur <= 0).sum())
dur.describe(percentiles=[0.01, 0.05, 0.5, 0.95, 0.99])

min 0.22998046875 max 13.5234375 mean 3.5650313478851627
<= 0 durations: 0


count    39248.000000
mean         3.565031
std          1.831246
min          0.229980
1%           0.729980
5%           1.333496
50%          3.279297
95%          6.988281
99%         10.566641
max         13.523438
Name: durationSeconds, dtype: float64

In [20]:
# Quick check that files exist (may take a few seconds)
paths = df["audio_wav_path"].astype(str)
exists_mask = paths.apply(os.path.exists)
print("Files found:", exists_mask.sum(), "of", len(paths))
missing_paths = df.loc[~exists_mask, ["id", "audio_wav_path"]]
missing_paths.head()

Files found: 39248 of 39248


,id,audio_wav_path


In [21]:
text_dups = (
    df.assign(text_trim=df["text"].str.strip())
      .groupby("text_trim")
      .agg(
          n_rows=("text_trim", "size"),
          n_unique_ids=("id", "nunique"),
          subsets=("subset", lambda x: x.unique()),
          styles=("style", lambda x: x.unique()),
          mics=("microphone", lambda x: x.unique())
      )
      .query("n_rows > 1")
      .sort_values("n_rows", ascending=False)
)

text_dups.head(5)



,n_rows,n_unique_ids,subsets,styles,mics
text_trim,,,,,
Zimtzicken und Muttersöhnchen.,9,9,"[TV-2021.02-Neutral, TV-2023.09-Hessisch, TV-2...","[neutral, Hessisch (german dialect), disgusted...",[good_rodePodcaster]
Hotel oder Ferienwohnung?,9,9,"[TV-2021.02-Neutral, TV-2023.09-Hessisch, TV-2...","[neutral, Hessisch (german dialect), sleepy | ...",[good_rodePodcaster]
Gute Radwege sind asphaltiert.,9,9,"[TV-2021.02-Neutral, TV-2023.09-Hessisch, TV-2...","[neutral, Hessisch (german dialect), amused | ...",[good_rodePodcaster]
"Herr, schmeiß Hirn vom Himmel.",9,9,"[TV-2021.02-Neutral, TV-2023.09-Hessisch, TV-2...","[neutral, Hessisch (german dialect), amused | ...",[good_rodePodcaster]
"Hundert Gigabyte, wer soll jemals so viel Speicherplatz brauchen?",9,9,"[TV-2021.02-Neutral, TV-2023.09-Hessisch, TV-2...","[neutral, Hessisch (german dialect), sleepy | ...",[good_rodePodcaster]


In [22]:
duplicate_texts = (
    df["text"].str.strip().value_counts()
    .loc[lambda x: x > 1]
    .index.tolist()
)

df_with_dups = df.assign(text_trim=df["text"].str.strip())
rows_with_dups = df_with_dups[df_with_dups["text_trim"].isin(duplicate_texts)]

rows_with_dups = rows_with_dups.sort_values("text_trim")
display(rows_with_dups.head(10))

,idx,id,subset,style,text,samplerate,durationSeconds,recording_year_month,microphone,language,comment,audio_wav_path,text_trim
9358,9358,4aeeae88-0777-2c8c-5c93-2e844a462e49---b74c74f...,TV-2021.02-Neutral,neutral,Ab wann gilt die Tätigkeit als gewerblich?,16000,2.859375,2019-10,bad_usbHeadset,german,NaN,/Volumes/SSanDisk/SpeechRec-German/data_wav/TV...,Ab wann gilt die Tätigkeit als gewerblich?
28413,28413,dd01c488-10f3-a683-00cf-4d215f4d9b19---b74c74f...,TV-2022.10-Neutral,neutral,Ab wann gilt die Tätigkeit als gewerblich?,16000,2.509766,2021-06,bad_usbHeadset,german,NaN,/Volumes/SSanDisk/SpeechRec-German/data_wav/TV...,Ab wann gilt die Tätigkeit als gewerblich?
8365,8365,4aeeae88-0777-2c8c-5c93-2e844a462e49---b36568c...,TV-2021.02-Neutral,neutral,Aber ab Mitternacht ist Schicht im Schacht.,16000,2.589844,2019-10,bad_usbHeadset,german,NaN,/Volumes/SSanDisk/SpeechRec-German/data_wav/TV...,Aber ab Mitternacht ist Schicht im Schacht.
30929,30929,dd01c488-10f3-a683-00cf-4d215f4d9b19---b36568c...,TV-2022.10-Neutral,neutral,Aber ab Mitternacht ist Schicht im Schacht.,16000,2.240234,2021-06,bad_usbHeadset,german,NaN,/Volumes/SSanDisk/SpeechRec-German/data_wav/TV...,Aber ab Mitternacht ist Schicht im Schacht.
13994,13994,4aeeae88-0777-2c8c-5c93-2e844a462e49---5cdd317...,TV-2021.02-Neutral,neutral,Aber das ist wohl der Situation geschuldet.,16000,2.550781,2019-10,bad_usbHeadset,german,NaN,/Volumes/SSanDisk/SpeechRec-German/data_wav/TV...,Aber das ist wohl der Situation geschuldet.
34074,34074,dd01c488-10f3-a683-00cf-4d215f4d9b19---5cdd317...,TV-2022.10-Neutral,neutral,Aber das ist wohl der Situation geschuldet.,16000,2.199219,2021-06,bad_usbHeadset,german,NaN,/Volumes/SSanDisk/SpeechRec-German/data_wav/TV...,Aber das ist wohl der Situation geschuldet.
28263,28263,dd01c488-10f3-a683-00cf-4d215f4d9b19---0287444...,TV-2022.10-Neutral,neutral,"Aber die Sicherheit steht und fällt damit, das...",16000,4.179688,2021-06,bad_usbHeadset,german,NaN,/Volumes/SSanDisk/SpeechRec-German/data_wav/TV...,"Aber die Sicherheit steht und fällt damit, das..."
8949,8949,4aeeae88-0777-2c8c-5c93-2e844a462e49---0287444...,TV-2021.02-Neutral,neutral,"Aber die Sicherheit steht und fällt damit, das...",16000,4.558594,2019-10,bad_usbHeadset,german,NaN,/Volumes/SSanDisk/SpeechRec-German/data_wav/TV...,"Aber die Sicherheit steht und fällt damit, das..."
14344,14344,4aeeae88-0777-2c8c-5c93-2e844a462e49---29acbfd...,TV-2021.02-Neutral,neutral,Aber die eigentliche Party steigt woanders.,16000,2.580078,2019-10,bad_usbHeadset,german,NaN,/Volumes/SSanDisk/SpeechRec-German/data_wav/TV...,Aber die eigentliche Party steigt woanders.
31101,31101,dd01c488-10f3-a683-00cf-4d215f4d9b19---29acbfd...,TV-2022.10-Neutral,neutral,Aber die eigentliche Party steigt woanders.,16000,2.230469,2021-06,bad_usbHeadset,german,NaN,/Volumes/SSanDisk/SpeechRec-German/data_wav/TV...,Aber die eigentliche Party steigt woanders.
